In [31]:
import numpy as np
import cma
import re
import math

x0 = 0.3
y0 = 0.3 
theta0 = math.pi
delta_t = 0.01

input_seq = [
                [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],
                [1,1],[1,1],[1,1],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0.3],[1,0.3],
                [1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[0.3,1],
                [0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1]
            ]

In [15]:
def generate_path(L, r):
    x = x0
    y = y0
    theta = thetha0

    path = [[x,y,theta]]

    for wl, wr in input_seq:
        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x + v * delta_t * math.cos(theta)
        y = y + v * delta_t * math.sin(theta)
        theta = theta + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        path.append([x,y,theta])
    
    return path

In [16]:
def loss(params):
    real = generate_path(real_params[0], real_params[1])
    pred = generate_path(params[0], params[1])
    return np.sqrt(np.mean((np.array(pred) - np.array(real)) ** 2))

In [17]:
def dist2target(path,target=[0,0]):
    return [np.linalg.norm(target)-np.array(step[0:-1])) for step in path]

In [18]:
L = 0.8
r = 0.69
real_params = [L,r]

L_fake = 0.6
r_fake = 0.13
fake_params = [L_fake,r_fake]

test_loss = loss(real_params)


In [19]:
es = cma.CMAEvolutionStrategy(len(real_params) * [0], 0.5, {'verbose': -3})
es.optimize(loss)

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 7.188720072169344e-02 1.0e+00 4.39e-01  3e-01  5e-01 0:00.0
    2     12 2.755491942895244e-02 1.4e+00 7.14e-01  6e-01  7e-01 0:00.0
    3     18 1.811749335266345e-02 1.3e+00 7.32e-01  6e-01  6e-01 0:00.0
  100    600 2.623307939999283e-13 4.7e+00 5.82e-08  2e-12  7e-12 0:00.3


In [20]:
print(f'Best loss value: {loss(es.result.xbest):.2}')

Best loss value: 1.1e-13


In [21]:
print('The parameters set is: L = {}, r = {}'.format(es.result.xbest[0],es.result.xbest[1]))


The parameters set is: L = 0.800000000001304, r = 0.6900000000009019


In [90]:
# Alternative with full control 
import numpy as np
import cma
import re
import math

x0 = 0.3
y0 = 0.3 
theta0 = math.pi
delta_t = 0.01

input_seq = [
                [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],
                [1,1],[1,1],[1,1],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0.3],[1,0.3],
                [1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[0.3,1],
                [0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1]
            ]

def generate_mpc_history(r, input_seq):
    x = x0
    y = y0
    theta = theta0

    hist = []
    elem = {'init_state':[x,y,theta]}
    for wl, wr in input_seq:
        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x + v * delta_t * math.cos(theta)
        y = y + v * delta_t * math.sin(theta)
        theta = theta + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        elem['input'] = [wl, wr]
        elem['end_state'] = [x,y,theta]
        hist.append(elem)
        elem = {'init_state':[x,y,theta]}
    return hist

def generate_step_history(r, mpc_history):
    hist = []
    for elem in mpc_history:
        x0, y0, theta0 = elem['init_state']
        wl, wr = elem['input']

        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x0 + v * delta_t * math.cos(theta0)
        y = y0 + v * delta_t * math.sin(theta0)
        theta = theta0 + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        hist.append([x,y,theta])
    return hist

def get_mean_sigma(param_set):
    min_param = min(param_set)
    max_param = max(param_set)

    mean = (min_param+max_param)/2
    sigma = (max_param-min_param)/4

    return mean, sigma

def loss(param_val):
    step_hist = generate_step_history(param_val[0], mpc_history)
    return np.sqrt(np.mean((np.array(step_mpc_history) - np.array(step_hist)) ** 2))

In [91]:
PARAMS_NUM = 1
SCENARIOS_NUM = 3

L = 0.5
r_set = [0.12, 0.16, 0.17]

true_r = 0.15

mean, sigma = get_mean_sigma(r_set)
print(mean, sigma)

0.14500000000000002 0.012500000000000004


In [95]:
step_mpc_history = [elem['end_state'] for elem in mpc_history]

es = cma.CMAEvolutionStrategy((PARAMS_NUM+1)*[mean], sigma, {'verbose': -3,'popsize': SCENARIOS_NUM})
while not es.stop():
    solutions = es.ask()
    es.tell(solutions, [loss(param_val) for param_val in solutions])
    es.disp(20)
print('terminated on ' + str(es.stop()))


Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      3 1.193135919841615e-04 1.0e+00 1.06e-02  1e-02  1e-02 0:00.0
    2      6 8.786813226335913e-05 1.1e+00 9.28e-03  8e-03  9e-03 0:00.0
    3      9 1.135274461730694e-05 1.1e+00 8.63e-03  7e-03  8e-03 0:00.0
   20     60 9.679734274089705e-06 1.6e+00 2.26e-03  1e-03  2e-03 0:00.0
   40    120 7.088562898379419e-07 3.4e+00 6.04e-04  1e-04  4e-04 0:00.1
   60    180 9.408830891815638e-07 6.3e+00 9.99e-04  1e-04  7e-04 0:00.1
   80    240 1.838893065804717e-07 1.5e+01 9.11e-04  5e-05  8e-04 0:00.1
  100    300 1.184672952634339e-07 1.9e+01 6.44e-04  2e-05  5e-04 0:00.1
  120    360 3.713658530571715e-08 3.8e+01 1.11e-04  2e-06  6e-05 0:00.2
  140    420 5.358547303055090e-10 7.0e+01 1.51e-05  1e-07  8e-06 0:00.2
  160    480 9.616081484609850e-11 1.0e+02 1.82e-06  6e-09  6e-07 0:00.2
  180    540 4.443956739249619e-11 9.5e+01 1.99e-06  6e-09  6e-07 0:00.3
  200    600 4.982890071099600e-01 1.1e+02 1.32e-06 

In [97]:
print('The parameters set is: r = {}, _ = {}'.format(es.result.xbest[0],es.result.xbest[1]))


The parameters set is: r = 0.14999999996607885, _ = 0.16157474311195627
